# Agenda

1. Iterators and the iterator protocol
    - The protocol itself
    - Making our classes iterable
    - Generators and generator functions
    - Generator expressions / generator comprehensions
    - `itertools`
2. Decorators
3. Concurrency
    - Threads
    - Multiprocessing
    - `asyncio`


# Iterators

We know that we can put many different data types into a `for` loop:

- Strings, and we get one character per iteration
- Lists or tuples, one element per iteration
- Dicts, one key per iteration
- Files, one line per iteration



In [1]:
for one_character in 'abcd':
    print(one_character)

a
b
c
d


# Iterator protocol

1. `for` turns to the object at the end of the line, and asks: Are you iterable? (`iter`)
    - If not, then we exit with a `TypeError` exception
2. If so, then we get an iteator object back, to which `for` says: Give me your next value (`next`)
    - If there are no more values, then the loop exits (becuse `next` raised the `StopIteration` exception)
3. The next value is assigned to the loop variable (`one_character`)
4. The loop body executes with that assignment
5. We return to step 2

In [2]:
s = 'abcd'

i = iter(s)
type(i)

str_ascii_iterator

In [3]:
next(i)

'a'

In [4]:
next(i)

'b'

In [5]:
next(i)

'c'

In [6]:
next(i)

'd'

In [7]:
next(i)

StopIteration: 

In [8]:
f = open('/etc/passwd')
i = iter(f)

In [9]:
i

<_io.TextIOWrapper name='/etc/passwd' mode='r' encoding='UTF-8'>

In [10]:
f is i   # is f its own iterator?

True

In [11]:
iter(10)

TypeError: 'int' object is not iterable

In [12]:
for i in 10:
    print(i)

TypeError: 'int' object is not iterable

In [16]:
class MyIter:
    def __init__(self, data):
        self.data = data
        self.index = 0
        print(f'\tIn MyIter.__init__, {self.data=}, {self.index=}')

    def __iter__(self):  # the job of __iter__ is to return the object's iterator, where __next__ is implemented
        print(f'\tIn MyIter.__iter__, {self.data=}, {self.index=}')
        return self      # I am my own iterator!

    def __next__(self):
        print(f'\tIn MyIter.__next__, {self.data=}, {self.index=}')
        if self.index >= len(self.data):
            print(f'\tIn MyIter.__next__, raising StopIteration')
            raise StopIteration     # stop the loop if we've gone past the end

        value = self.data[self.index]
        self.index += 1
        print(f'\tIn MyIter.__next__, returning {value}')
        return value

m = MyIter('abcd')

print('*** First run')
for one_item in m:
    print(one_item)

print('*** Second run')
for one_item in m:
    print(one_item)

	In MyIter.__init__, self.data='abcd', self.index=0
*** First run
	In MyIter.__iter__, self.data='abcd', self.index=0
	In MyIter.__next__, self.data='abcd', self.index=0
	In MyIter.__next__, returning a
a
	In MyIter.__next__, self.data='abcd', self.index=1
	In MyIter.__next__, returning b
b
	In MyIter.__next__, self.data='abcd', self.index=2
	In MyIter.__next__, returning c
c
	In MyIter.__next__, self.data='abcd', self.index=3
	In MyIter.__next__, returning d
d
	In MyIter.__next__, self.data='abcd', self.index=4
	In MyIter.__next__, raising StopIteration
*** Second run
	In MyIter.__iter__, self.data='abcd', self.index=4
	In MyIter.__next__, self.data='abcd', self.index=4
	In MyIter.__next__, raising StopIteration


# Exercise: Circle

1. Write a `Circle` class that takes two arguments, an iterable (`data`) and an integer (`maxtimes`).
2. When someone iterates over an instance of `Circle`, they should get `maxtimes` results.
3. If `maxtimes` is bigger than the length of `Circle`, then you should go back to the beginning for more values.

Example:

```python
c = Circle('abcd', 7)

for one_item in c:
    print(one_item)  # a b c d a b c 
```    

In [ ]:
class Circle:
    def __init__(self, data, maxtimes):
        self.data = data
        self.maxtimes = maxtimes
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index >= len(self.data):
            raise StopIteration

        value = self.data[self.index]
        self.index += 1
        return 